# Práctica
> Angel Luis Valdés Sánchez

Utilizando una herramienta o lenguaje de programación, realice lo
siguiente:
1. Descargar un conjunto de datos que contenga datos faltantes del
repositorio UCI.
2. Identificar los datos faltantes y aplicar las ténicas descritas
anteriormente para rellenar la matriz de datos.
3. Elaborar un reporte en una libreta de Jupyter Notebook.

In [111]:
#pip install ucimlrepo

In [112]:
import pandas as pd
from ucimlrepo import fetch_ucirepo 

# Obtener el dataset de mushrooms desde UCI
mushrooms = fetch_ucirepo(id=73) 

X = mushrooms.data.features 
y = mushrooms.data.targets 

data = pd.concat([X, y], axis=1)

print(f"Forma del dataset: {data.shape}")

print("\nPrimeras 5 filas:")
data.head()


Forma del dataset: (8124, 23)

Primeras 5 filas:


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,poisonous
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


## Analizar datos faltantes

In [113]:
# Analizar los datos faltantes por columna en el DataFrame 'data'
missing_counts = data.isnull().sum()
missing_percent = (missing_counts / len(data)) * 100

missing_summary = pd.DataFrame({
    'Columna': data.columns,
    'Valores_Faltantes': missing_counts.values,
    'Porcentaje': missing_percent.values
})

missing_summary

,Columna,Valores_Faltantes,Porcentaje
0,cap-shape,0,0.000000
1,cap-surface,0,0.000000
2,cap-color,0,0.000000
3,bruises,0,0.000000
4,odor,0,0.000000
5,gill-attachment,0,0.000000
6,gill-spacing,0,0.000000
7,gill-size,0,0.000000
8,gill-color,0,0.000000
9,stalk-shape,0,0.000000


## Técnicas de Limpieza de Datos Faltantes

Ahora aplicaremos las diferentes técnicas para manejar los datos faltantes:

1. **Ignorar tuplas con datos faltantes**
2. **Rellenar con constante global**
3. **Rellenar con medidas de tendencia central (media/mediana)**
4. **Rellenar con media/mediana por clase**

### 1. Ignorar tuplas con datos faltantes

In [135]:
# Técnica 1: Ignorar tuplas con datos faltantes (con umbral inteligente)

# Analizar distribución de valores faltantes por fila
missing_per_row = data.isnull().sum(axis=1)
missing_percent_per_row = (missing_per_row / data.shape[1]) * 100

print("=== ANÁLISIS DE FILAS CON DATOS FALTANTES ===")
print(f"Filas sin datos faltantes: {(missing_per_row == 0).sum()}")
print(f"Filas con 1-25% de datos faltantes: {((missing_percent_per_row > 0) & (missing_percent_per_row <= 25)).sum()}")
print(f"Filas con > 26% de datos faltantes: {((missing_percent_per_row > 25)).sum()}")

# Estrategia 2: Eliminar filas solo si tienen más del 5% de valores faltantes
threshold = 0.01
data_drop_threshold = data[missing_percent_per_row <= threshold * 100]
print(f"\nEliminar filas con > 1% de valores faltantes:")
print(f"   Filas eliminadas: {data.shape[0] - data_drop_threshold.shape[0]} ({((data.shape[0] - data_drop_threshold.shape[0]) / data.shape[0]) * 100:.1f}%)")
print(f"   Filas restantes: {data_drop_threshold.shape[0]}")
print(f"   Valores faltantes restantes: {data_drop_threshold.isnull().sum().sum()}")


=== ANÁLISIS DE FILAS CON DATOS FALTANTES ===
Filas sin datos faltantes: 5644
Filas con 1-25% de datos faltantes: 2480
Filas con > 26% de datos faltantes: 0

Eliminar filas con > 1% de valores faltantes:
   Filas eliminadas: 2480 (30.5%)
   Filas restantes: 5644
   Valores faltantes restantes: 0


### 2. Rellenar con constante global

In [138]:
# Técnica 2: Rellenar con constante global (usando valores estándar de Python)
import numpy as np

# Definir constantes estándar para diferentes tipos de datos
UNKNOWN_CATEGORICAL = 'UNK'  # Para variables categóricas
MISSING_NUMERICAL = -999     # Para variables numéricas enteras
MISSING_FLOAT = np.inf       # Para variables float (infinity)
MISSING_BOOLEAN = False      # Para variables booleanas

data_global_constant = data.copy()

print("\n=== PROCESANDO COLUMNAS ===")

for column in data_global_constant.columns:
    original_missing = data_global_constant[column].isnull().sum()
    
    if original_missing > 0:
        if data_global_constant[column].dtype in ['object', 'category']:
            # Variables categóricas/texto
            data_global_constant[column] = data_global_constant[column].fillna(UNKNOWN_CATEGORICAL)
            print(f"✓ {column} (categórica): {original_missing} valores → '{UNKNOWN_CATEGORICAL}'")
            
        elif data_global_constant[column].dtype == 'bool':
            # Variables booleanas
            data_global_constant[column] = data_global_constant[column].fillna(MISSING_BOOLEAN)
            print(f"✓ {column} (booleana): {original_missing} valores → {MISSING_BOOLEAN}")
            
        elif data_global_constant[column].dtype in ['int64', 'int32', 'int16', 'int8']:
            # Variables enteras
            # Convertir a float primero para poder usar np.inf si es necesario
            if data_global_constant[column].max() < 1000:  # Si los valores son pequeños
                data_global_constant[column] = data_global_constant[column].fillna(MISSING_NUMERICAL)
                print(f"✓ {column} (entero): {original_missing} valores → {MISSING_NUMERICAL}")
            else:
                # Para enteros grandes, convertir a float y usar inf
                data_global_constant[column] = data_global_constant[column].astype(float).fillna(MISSING_FLOAT)
                print(f"✓ {column} (entero→float): {original_missing} valores → {MISSING_FLOAT}")
                
        elif data_global_constant[column].dtype in ['float64', 'float32']:
            # Variables float
            data_global_constant[column] = data_global_constant[column].fillna(MISSING_FLOAT)
            print(f"✓ {column} (float): {original_missing} valores → {MISSING_FLOAT}")
            
        else:
            # Tipo de dato no reconocido, usar valor por defecto
            data_global_constant[column] = data_global_constant[column].fillna(UNKNOWN_CATEGORICAL)
            print(f"? {column} (desconocido): {original_missing} valores → '{UNKNOWN_CATEGORICAL}'")

print(f"\n=== RESULTADOS ===")
print(f"Valores faltantes después de rellenar con constantes globales: {data_global_constant.isnull().sum().sum()}")

# Mostrar estadísticas de los valores utilizados
print("\n=== ESTADÍSTICAS DE VALORES UTILIZADOS ===")
for column in data_global_constant.columns:
    if data_global_constant[column].dtype in ['object', 'category']:
        unk_count = (data_global_constant[column] == UNKNOWN_CATEGORICAL).sum()
        if unk_count > 0:
            print(f"  {column}: {unk_count} valores 'UNK' ({unk_count/len(data_global_constant)*100:.1f}%)")
    
    elif data_global_constant[column].dtype in ['float64', 'float32']:
        inf_count = np.isinf(data_global_constant[column]).sum()
        minus999_count = (data_global_constant[column] == MISSING_NUMERICAL).sum()
        if inf_count > 0:
            print(f"  {column}: {inf_count} valores 'inf' ({inf_count/len(data_global_constant)*100:.1f}%)")
        if minus999_count > 0:
            print(f"  {column}: {minus999_count} valores '-999' ({minus999_count/len(data_global_constant)*100:.1f}%)")
    
    elif data_global_constant[column].dtype in ['int64', 'int32', 'int16', 'int8']:
        minus999_count = (data_global_constant[column] == MISSING_NUMERICAL).sum()
        if minus999_count > 0:
            print(f"  {column}: {minus999_count} valores '-999' ({minus999_count/len(data_global_constant)*100:.1f}%)")


=== PROCESANDO COLUMNAS ===
✓ stalk-root (categórica): 2480 valores → 'UNK'

=== RESULTADOS ===
Valores faltantes después de rellenar con constantes globales: 0

=== ESTADÍSTICAS DE VALORES UTILIZADOS ===
  stalk-root: 2480 valores 'UNK' (30.5%)


In [141]:
# Técnica 3: Rellenar con medidas de tendencia central
data_central_tendency = data.copy()

# Para variables numéricas, usar media o mediana
# Para variables categóricas, usar la moda
for column in data_central_tendency.columns:
    if data_central_tendency[column].dtype in ['object', 'category']:
        # Usar la moda (valor más frecuente) para variables categóricas
        mode_value = data_central_tendency[column].mode()
        if len(mode_value) > 0:
            data_central_tendency[column] = data_central_tendency[column].fillna(mode_value[0])
    else:
        # Usar la mediana para variables numéricas (más robusta ante outliers)
        median_value = data_central_tendency[column].median()
        data_central_tendency[column] = data_central_tendency[column].fillna(median_value)

print(f"Valores faltantes después de rellenar con medidas de tendencia central: {data_central_tendency.isnull().sum().sum()}")
print("\nComparación de algunas columnas:")
print("Original vs Relleno con tendencia central:")
for col in ['carbon', 'hardness', 'stalk-root']:  # Ejemplos de columnas
    if col in data.columns:
        orig_missing = data[col].isnull().sum()
        if orig_missing > 0:
            print(f"{col}: {orig_missing} valores faltantes -> 0 valores faltantes")

Valores faltantes después de rellenar con medidas de tendencia central: 0

Comparación de algunas columnas:
Original vs Relleno con tendencia central:
stalk-root: 2480 valores faltantes -> 0 valores faltantes


### 4. Rellenar con media/mediana por clase

In [117]:
# Técnica 4: Rellenar con media/mediana por clase
data_class_based = data.copy()

# Usar la columna de clase (target) si existe
target_column = y.columns[0] if len(y.columns) > 0 else None

if target_column and target_column in data_class_based.columns:
    print(f"Usando la columna '{target_column}' como clase para agrupar")
    
    # Rellenar valores faltantes por clase
    for column in data_class_based.columns:
        if column != target_column and data_class_based[column].isnull().sum() > 0:
            if data_class_based[column].dtype in ['object', 'category']:
                # Para categóricas, usar la moda por clase
                data_class_based[column] = data_class_based.groupby(target_column)[column].transform(
                    lambda x: x.fillna(x.mode()[0] if len(x.mode()) > 0 else 'Desconocido')
                )
            else:
                # Para numéricas, usar la mediana por clase
                data_class_based[column] = data_class_based.groupby(target_column)[column].transform(
                    lambda x: x.fillna(x.median())
                )
    
    print(f"Valores faltantes después de rellenar por clase: {data_class_based.isnull().sum().sum()}")
else:
    print("No se encontró columna de clase válida. Usando relleno general.")
    data_class_based = data_central_tendency.copy()

Usando la columna 'poisonous' como clase para agrupar
Valores faltantes después de rellenar por clase: 0


## Conclusiones

### Resumen de Técnicas Aplicadas:

1. **Eliminar filas con datos faltantes**: Perdemos información valiosa pero obtenemos un dataset completamente limpio.

2. **Constante global**: Mantiene todos los datos pero introduce valores artificiales que pueden afectar los análisis.

3. **Tendencia central**: Usa la mediana/moda para rellenar, manteniendo la distribución general de los datos.

4. **Por clase**: Más sofisticado, usa estadísticas específicas por grupo para un relleno más contextual.

5. **Predicción**: La técnica más avanzada, utiliza machine learning para predecir valores basándose en otros atributos.

### Recomendaciones:

- **Para análisis exploratorio**: Usar técnicas 3 o 4 (tendencia central o por clase)
- **Para modelos de machine learning**: Considerar técnica 5 (predicción) o técnica 4 (por clase)
- **Para análisis donde la integridad de datos es crítica**: Técnica 1 (eliminar filas) si la pérdida de datos es aceptable